In [ ]:
import os
import sys
import time as t_util
import numpy as np
import cftime
import xarray as xr
import matplotlib.pyplot as plt
import scipy.stats
import matplotlib
import yaml


In [ ]:
#Read main paths
with open('../path_main.txt', 'r') as file:    path_main  = file.read()
with open('../path_gwls.txt', 'r') as file:    path_gwls  = file.read()
    
dir_CMIP5    = f'{path_main}Data/CMIP5/HWMId/'
dir_GWL      = f'{path_gwls}cmip5_all_ens/'
dir_EMT      = f'{path_main}Data/CMIP5/EMT/'
dir_names    = f'{path_main}Scripts/Model_lists/'
dir_out      = f'{path_main}Data/Plot_preparation/HWMId/CMIP5/'
if not os.path.exists(dir_out): os.mkdir(dir_out)


## Prepare variables and parameters

In [ ]:
#Define cities
cities = ['Lisbon', 'Madrid', 'Barcelona', 'Rome', 'Athens', 'Istanbul', 'Sofia', 'Bucharest', 'Belgrade', 'Zagreb',
          'Milan', 'Budapest', 'Munich', 'Vienna', 'Prague', 'Paris', 'Brussels', 'Amsterdam', 'London', 'Dublin',
          'Hamburg', 'Copenhagen', 'Berlin', 'Warsaw', 'Kharkiv', 'Kyiv', 'Minsk', 'Vilnius', 'Riga', 'Moscow',
          'NizhnyNovgorod', 'Kazan', 'SaintPetersburg', 'Helsinki', 'Stockholm', 'Oslo']

#Define HSIs
HSI_names = ['TX']

#Define models and RCPs which should be used
all_models = dict()
all_models['rcp85'] = []
with open(dir_names + 'Models_CMIP5_RCP85.txt', 'r') as filehandle:
    for line in filehandle:
        all_models['rcp85'].append(line[:-1])


#Get model names (including member)
dir_data = dir_CMIP5 + 'HWMId-TX/'
model_names = sorted(list(set(["_".join(file.split('_')[1:3]) for file in os.listdir(dir_data)])))

#Read warming levels
fname = dir_GWL + 'cmip5_warming_levels_all_ens_1850_1900_no_bounds_check.yml'
with open(fname, 'r') as file:
    GWL_data = yaml.safe_load(file)
    
#Define warming levels
GWL_levels = ['10', '20', '30']


## Prepare GWL data

In [ ]:
#Define RCP
RCP = 'rcp85'

quantiles = ['Q50', 'Q90']

#Loop over cities
for HSI in HSI_names:
    
    print(HSI)

    #Select file name
    dir_CMIP5_HWMId = dir_CMIP5 + 'HWMId-' + HSI + '/'

    #Create empty array for storing SREX-averaged heat indices
    da_empty1 = np.zeros((len(model_names), len(cities), len(GWL_levels), len(quantiles))) * np.NaN
    data_coll = xr.Dataset(coords={'model':     ('model', model_names),
                                   'city':      ('city', cities),
                                   'GWL_level': ('GWL_level', GWL_levels),
                                   'quantile': ('quantile', quantiles)})
    data_coll['HWMID'] = (('model', 'city', 'GWL_level', 'quantile'), da_empty1)    

    #Loop over cities
    for city in cities:
        
        #Loop over models
        for i1, model in enumerate(all_models[RCP]):

            #Define members
            if model=='EC-EARTH':      members = ['r1i1p1', 'r12i1p1']
            elif model=='MPI-ESM-LR':  members = ['r1i1p1', 'r2i1p1', 'r3i1p1']
            else:                      members = ['r1i1p1']        

            #Loop over members
            for member in members:

                #Define name for storing data in array
                model_name = model + '_' + member
            
                #Read data
                fname_CMIP5 = [file for file in os.listdir(dir_CMIP5_HWMId) if HSI in file and RCP + '_' in file and model + '_' in file and member in file]
                if len(fname_CMIP5)!=1:  sys.exit('File is not unique')
                data_CMIP5 = xr.open_dataset(dir_CMIP5_HWMId + fname_CMIP5[0])              

                #Read time periods when certain global warming levels (GWL) are reached
                time_GWL = dict()
                for level in GWL_levels:
                    data_level = GWL_data['warming_level_' + level]
                    entry_sel = [entry for entry in data_level if entry['model']==model and entry['exp']==RCP and entry['ensemble']==member]

                    #Select data in time period when GWL is reached
                    if len(entry_sel)==1:
                        time_GWL = slice(str(entry_sel[0]['start_year']), str(entry_sel[0]['end_year']))
                    else:
                        if city==cities[0]:  print(str(int(level)/10) + 'K not reached in ' + model)
                        continue              


                    #Calculate median and 90th quantile
                    data_Q50 = data_CMIP5.HWMID.sel(city=city, time=time_GWL).median('time')
                    data_Q90 = data_CMIP5.HWMID.sel(city=city, time=time_GWL).quantile(0.90, dim='time')

                    #Put data in array
                    data_coll.HWMID.loc[{"model": model_name, "city": city, "GWL_level": level, "quantile": 'Q50'}] = data_Q50
                    data_coll.HWMID.loc[{"model": model_name, "city": city, "GWL_level": level, "quantile": 'Q90'}] = data_Q90
                
    #Save data in file
    fname_out = dir_out + 'HWMId-' + HSI + '_' + RCP + '_GWL.nc'
    if os.path.exists(fname_out): os.remove(fname_out)
    data_coll.to_netcdf(fname_out)
  

## Prepare data according to European warming (EMT)

In [ ]:
#Define RCP
RCP = 'rcp85'

quantiles = ['Q50', 'Q90']

#Define changes in EMT relative to 1981-2010
EMT_change     = np.array([1.0, 2.0, 3.0])
EMT_change_str = ['1.0K', '2.0K', '3.0K']

#Loop over cities
for HSI in HSI_names:
    
    print(HSI)

    #Select file name
    dir_CMIP5_HWMId = dir_CMIP5 + 'HWMId-' + HSI + '/'

    #Create empty array for storing SREX-averaged heat indices
    da_empty1 = np.zeros((len(model_names), len(cities), len(EMT_change_str), len(quantiles))) * np.NaN
    data_coll = xr.Dataset(coords={'model':     ('model', model_names),
                                   'city':      ('city', cities),
                                   'EMT_change': ('EMT_change', EMT_change_str),
                                   'quantile': ('quantile', quantiles)})
    data_coll['HWMID'] = (('model', 'city', 'EMT_change', 'quantile'), da_empty1)

    #Loop over cities
    for city in cities:
        
        #Loop over models
        for i1, model in enumerate(all_models[RCP]):

            #Define members
            if model=='EC-EARTH':      members = ['r1i1p1', 'r12i1p1']
            elif model=='MPI-ESM-LR':  members = ['r1i1p1', 'r2i1p1', 'r3i1p1']
            else:                      members = ['r1i1p1']        

            #Loop over members
            for member in members:

                #Define name for storing data in array
                model_name = model + '_' + member

                #Read data
                fname_CMIP5 = [file for file in os.listdir(dir_CMIP5_HWMId) if HSI in file and RCP + '_' in file and model + '_' in file and member in file]
                if len(fname_CMIP5)!=1:  sys.exit('File is not unique')
                data_CMIP5 = xr.open_dataset(dir_CMIP5_HWMId + fname_CMIP5[0])              

                #Read European mean temperature (EMT)
                files_EMT = sorted([dir_EMT + file for file in os.listdir(dir_EMT) if model + '_' in file and 'EMT_' in file and member in file])
                data_EMT  = xr.concat((xr.open_dataset(file) for file in files_EMT), dim='time')

                #Calculate EMT relative to 1981-2010 and calculate 20-year means
                dataEMT_ref = data_EMT.sel(time=slice('1981', '2010')).mean('time')
                dataEMT_rel = data_EMT - dataEMT_ref
                dataEMT_20y = dataEMT_rel.rolling(time=20, center=True).mean()

                #Loop over selected EMT levels
                for dEMT, dEMT_str in zip(EMT_change, EMT_change_str):

                    #Identify 20-year period in wich level is reached for first time
                    dEMT_ind  = np.where(dataEMT_20y.tas>dEMT)
                    if len(dEMT_ind[0])==0:
                        if city==cities[0]:  print("  " + dEMT_str + ' not reached in ' + model)
                        continue

                    ind = dEMT_ind[0][0]
                    central_year = dataEMT_20y.isel(time=ind).time.dt.year
                    start_year   = int(central_year - 20 / 2)
                    end_year     = int(central_year + (20 / 2 - 1))
                    years_sel    = slice(str(start_year), str(end_year))

                    if end_year>2099:
                        print(model)
                        print(end_year)

                    #Calculate median and 90th quantile
                    data_Q50 = data_CMIP5.HWMID.sel(city=city, time=years_sel).median('time')
                    data_Q90 = data_CMIP5.HWMID.sel(city=city, time=years_sel).quantile(0.90, dim='time')

                    #Put data in array
                    data_coll.HWMID.loc[{"model": model_name, "city": city, "EMT_change": dEMT_str, "quantile": 'Q50'}] = data_Q50
                    data_coll.HWMID.loc[{"model": model_name, "city": city, "EMT_change": dEMT_str, "quantile": 'Q90'}] = data_Q90

    #Save data in file
    fname_out = dir_out + 'HWMId-' + HSI + '_' + RCP + '_EMT.nc'
    if os.path.exists(fname_out): os.remove(fname_out)
    data_coll.to_netcdf(fname_out)
  

## Prepare time period data

In [ ]:
#Select time periods
time_periods = [[1981, 2010],
                [2036, 2065],
                [2070, 2099]]

#Define time string
time_strings = [str(time[0]) + '-' + str(time[1]) for time in time_periods]

#Define RCPs
RCPs = ['rcp85']

quantiles = ['Q50', 'Q90']

#Loop over RCPs
for RCP in RCPs:

    #Loop over cities
    for HSI in HSI_names:
        
        print(HSI)
            
        #Select folder
        dir_CMIP5_HWMId = dir_CMIP5 + 'HWMId-' + HSI + '/'

        #Create empty array for storing SREX-averaged heat indices
        da_empty1 = np.zeros((len(model_names), len(cities), len(time_periods), len(quantiles))) * np.NaN
        data_coll = xr.Dataset(coords={'model':       ('model', model_names),
                                       'city':        ('city', cities),
                                       'time_period': ('time_period', time_strings),
                                       'quantile': ('quantile', quantiles)})
        data_coll['HWMID'] = (('model', 'city', 'time_period', 'quantile'), da_empty1)    

        #Loop over cities
        for city in cities:
            
            #Loop over models
            for i1, model in enumerate(all_models[RCP]):

                #Define members
                if model=='EC-EARTH':      members = ['r1i1p1', 'r12i1p1']
                elif model=='MPI-ESM-LR':  members = ['r1i1p1', 'r2i1p1', 'r3i1p1']
                else:                      members = ['r1i1p1']        

                #Loop over members
                for member in members:

                    #Define name for storing data in array
                    model_name = model + '_' + member
                    
                    #Read data
                    fname_CMIP5 = [file for file in os.listdir(dir_CMIP5_HWMId) if HSI in file and RCP + '_' in file and model + '_' in file and member in file]
                    if len(fname_CMIP5)!=1:  sys.exit('File is not unique')
                    data_CMIP5 = xr.open_dataset(dir_CMIP5_HWMId + fname_CMIP5[0])

                    #Loop over time periods
                    for time_period, time_string in zip(time_periods, time_strings):

                        #Select data in time period when GWL is reached
                        time_sel = slice(str(time_period[0]), str(time_period[1]))

                        data_Q50 = data_CMIP5.sel(city=city, time=time_sel).median('time')
                        data_Q90 = data_CMIP5.sel(city=city, time=time_sel).quantile(0.90, dim='time')

                        #Put data in array
                        data_coll.HWMID.loc[{"model": model_name, "city": city, "time_period": time_string, "quantile": 'Q50'}] = data_Q50.HWMID
                        data_coll.HWMID.loc[{"model": model_name, "city": city, "time_period": time_string, "quantile": 'Q90'}] = data_Q90.HWMID

            #Save data in file
            fname_out = dir_out + 'HWMId-' + HSI + '_' + RCP + '_time-periods.nc'
            if os.path.exists(fname_out): os.remove(fname_out)
            data_coll.to_netcdf(fname_out)
